In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 용산구 투기지역/흡연구역 연관성 EDA

In [2]:
df_t = pd.read_csv('https://raw.githubusercontent.com/ecothonCBB/CigaretteButtBins/main/data/%E1%84%89%E1%85%A5%E1%84%8B%E1%85%AE%E1%86%AF%E1%84%90%E1%85%B3%E1%86%A8%E1%84%87%E1%85%A7%E1%86%AF%E1%84%89%E1%85%B5%20%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%89%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AE_%E1%84%83%E1%85%A1%E1%86%B7%E1%84%87%E1%85%A2%E1%84%81%E1%85%A9%E1%86%BC%E1%84%8E%E1%85%A9%E1%84%89%E1%85%A1%E1%86%BC%E1%84%89%E1%85%B3%E1%86%B8%E1%84%86%E1%85%AE%E1%84%83%E1%85%A1%E1%86%AB%E1%84%90%E1%85%AE%E1%84%80%E1%85%B5%E1%84%8C%E1%85%B5%E1%84%8B%E1%85%A7%E1%86%A8%E1%84%92%E1%85%A7%E1%86%AB%E1%84%92%E1%85%AA%E1%86%BC_20220818.csv')
df_t.head()

,연번,도로명주소,지번주소,상세위치
0,1,NaN,서울특별시 용산구 동자동 8-3,새꿈어린이공원
1,2,서울특별시 용산구 한강대로77길 12,서울특별시 용산구 갈월동 93-30,다이소 남영역점 옆
2,3,서울특별시 용산구 한강대로77길 13,서울특별시 용산구 갈월동 93-35,본투비 남영본점 옆
3,4,서울특별시 용산구 한강대로 100,서울특별시 용산구 한강로2가 424,아모레퍼시픽그룹 본사 뒤
4,5,서울특별시 용산구 백범로 341,서울특별시 용산구 원효로1가 133-3,원효1동주민센터 맞은편 아파트


In [4]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   연번      6 non-null      int64 
 1   도로명주소   5 non-null      object
 2   지번주소    6 non-null      object
 3   상세위치    6 non-null      object
dtypes: int64(1), object(3)
memory usage: 320.0+ bytes


In [5]:
df_a = pd.read_csv('https://raw.githubusercontent.com/ecothonCBB/CigaretteButtBins/main/data/%E1%84%89%E1%85%A5%E1%84%8B%E1%85%AE%E1%86%AF%E1%84%90%E1%85%B3%E1%86%A8%E1%84%87%E1%85%A7%E1%86%AF%E1%84%89%E1%85%B5%20%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%89%E1%85%A1%E1%86%AB%E1%84%80%E1%85%AE_%E1%84%92%E1%85%B3%E1%86%B8%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%80%E1%85%AE%E1%84%8B%E1%85%A7%E1%86%A8_20230316.csv', encoding='CP949')
df_a.head()

,자치구명,시설 구분,시설형태,서울특별시 용산구 설치 위치,설치 주체,위도,경도
0,용산구,철도역,개방형,서울특별시 용산구 서울역 광장 15번출구,한국철도공사,37.553149,126.968881
1,용산구,철도역,개방형,서울특별시 용산구 서울역 광장 1번출구,한국철도공사,37.553760,126.969662
2,용산구,철도역(조례),개방형,서울특별시 용산구 용산역 광장,현대아이파크몰,37.528404,126.965569
3,용산구,청사,개방형,서울특별시 용산구 용산구청 옥상,용산구청,37.532709,126.990000
4,용산구,청사,개방형,서울특별시 용산구 용산구청 2층 외부,용산구청,37.532709,126.990000


In [6]:
print(df_a.info())
df_a.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   자치구명             72 non-null     object 
 1   시설 구분            72 non-null     object 
 2   시설형태             72 non-null     object 
 3   서울특별시 용산구 설치 위치  72 non-null     object 
 4   설치 주체            72 non-null     object 
 5   위도               72 non-null     float64
 6   경도               72 non-null     float64
dtypes: float64(2), object(5)
memory usage: 4.1+ KB
None


(72, 7)

In [ ]:
import folium

# 서울시의 위도
latitude = 37.5665350
longitude = 126.9779692

seoul = folium.Map(location=[latitude, longitude],
               zoom_start=12, 
               width=1300, 
               height=800
              )
seoul

In [10]:
import json
import requests

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + 'key값'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']), float(match_first['x'])

address= df_t['지번주소']
print(address.head())

latitude = []
longitude =[]

for i in address:
    try:
        result = addr_to_lat_lon(i)
        latitude.append(result[0])
        longitude.append(result[1])
    except Exception as e:
        print(f"주소 검색 오류: {e}")
        latitude.append(None)  # 또는 다른 기본값 설정
        longitude.append(None)

df_t['위도'] = latitude
df_t['경도'] = longitude
df_t.head()

0        서울특별시 용산구 동자동 8-3
1      서울특별시 용산구 갈월동 93-30
2      서울특별시 용산구 갈월동 93-35
3      서울특별시 용산구 한강로2가 424
4    서울특별시 용산구 원효로1가 133-3
Name: 지번주소, dtype: object
주소 검색 오류: list index out of range


,연번,도로명주소,지번주소,상세위치,위도,경도
0,1,NaN,서울특별시 용산구 동자동 8-3,새꿈어린이공원,37.552414,126.974268
1,2,서울특별시 용산구 한강대로77길 12,서울특별시 용산구 갈월동 93-30,다이소 남영역점 옆,37.542219,126.971865
2,3,서울특별시 용산구 한강대로77길 13,서울특별시 용산구 갈월동 93-35,본투비 남영본점 옆,NaN,NaN
3,4,서울특별시 용산구 한강대로 100,서울특별시 용산구 한강로2가 424,아모레퍼시픽그룹 본사 뒤,37.528852,126.968617
4,5,서울특별시 용산구 백범로 341,서울특별시 용산구 원효로1가 133-3,원효1동주민센터 맞은편 아파트,37.538277,126.967363


In [11]:
# NaN값 위경도 찾아서 직접 기입(1행 위경도와 동일)

df_t['위도'][2] = df_t['위도'][1]
df_t['경도'][2] = df_t['경도'][1]
df_t

<ipython-input-11-3d3b13496821>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['위도'][2] = df_t['위도'][1]
<ipython-input-11-3d3b13496821>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_t['경도'][2] = df_t['경도'][1]


,연번,도로명주소,지번주소,상세위치,위도,경도
0,1,NaN,서울특별시 용산구 동자동 8-3,새꿈어린이공원,37.552414,126.974268
1,2,서울특별시 용산구 한강대로77길 12,서울특별시 용산구 갈월동 93-30,다이소 남영역점 옆,37.542219,126.971865
2,3,서울특별시 용산구 한강대로77길 13,서울특별시 용산구 갈월동 93-35,본투비 남영본점 옆,37.542219,126.971865
3,4,서울특별시 용산구 한강대로 100,서울특별시 용산구 한강로2가 424,아모레퍼시픽그룹 본사 뒤,37.528852,126.968617
4,5,서울특별시 용산구 백범로 341,서울특별시 용산구 원효로1가 133-3,원효1동주민센터 맞은편 아파트,37.538277,126.967363
5,6,서울특별시 용산구 한강대로 186,서울특별시 용산구 한강로1가 137-1,평양집 안쪽 골목,37.534879,126.973988


### 🔴 투기지역, 🔵 흡연구역

In [12]:
map_info = df_a[["위도", "경도"]].dropna()
for lat, long in zip(map_info["위도"], map_info["경도"]):
    folium.CircleMarker([lat, long], radius=23, color='#3186cc', fill_color='#3186cc').add_to(seoul)

map_info2 = df_t[["위도", "경도"]].dropna()
for lat, long in zip(map_info2["위도"], map_info2["경도"]):
    folium.CircleMarker([lat, long], radius=50, color='#cc4831', fill_color='#cc4831').add_to(seoul)

seoul

# 마포구 투기지역/흡연구역 연관성 EDA

In [13]:
mf_t = pd.read_csv('https://raw.githubusercontent.com/ecothonCBB/CigaretteButtBins/main/data/%E1%84%89%E1%85%A5%E1%84%8B%E1%85%AE%E1%86%AF%E1%84%90%E1%85%B3%E1%86%A8%E1%84%87%E1%85%A7%E1%86%AF%E1%84%89%E1%85%B5%20%E1%84%86%E1%85%A1%E1%84%91%E1%85%A9%E1%84%80%E1%85%AE%20%E1%84%83%E1%85%A1%E1%86%B7%E1%84%87%E1%85%A2%E1%84%81%E1%85%A9%E1%86%BC%E1%84%8E%E1%85%A9%20%E1%84%89%E1%85%A1%E1%86%BC%E1%84%89%E1%85%B3%E1%86%B8%20%E1%84%86%E1%85%AE%E1%84%83%E1%85%A1%E1%86%AB%E1%84%90%E1%85%AE%E1%84%80%E1%85%B5%20%E1%84%8C%E1%85%B5%E1%86%B8%E1%84%8C%E1%85%AE%E1%86%BC%E1%84%80%E1%85%AA%E1%86%AB%E1%84%85%E1%85%B5%20%E1%84%8C%E1%85%B5%E1%84%8B%E1%85%A7%E1%86%A8_20220805.csv')
mf_t.drop(['Unnamed: 5'], axis=1, inplace=True)
mf_t

,시도명,시군구명,도로명주소,세부위치,데이터기준일자
0,서울특별시,마포구,서울특별시 마포구 양화로 156,홍대입구역 9번출구 LG팰리스 뒤쪽 도로,2022-08-05
1,서울특별시,마포구,서울특별시 마포구 양화로 183,홍대입구역 3번출구 인근,2022-08-05
2,서울특별시,마포구,서울특별시 마포구 마포대로 144,마포T타운 뒤쪽 도로,2022-08-05
3,서울특별시,마포구,서울특별시 마포구 상암산로 38,kbs 건물 인근,2022-08-05


In [14]:
mf_a = pd.read_csv('https://raw.githubusercontent.com/ecothonCBB/CigaretteButtBins/main/data/%E1%84%89%E1%85%A5%E1%84%8B%E1%85%AE%E1%86%AF%E1%84%90%E1%85%B3%E1%86%A8%E1%84%87%E1%85%A7%E1%86%AF%E1%84%89%E1%85%B5%20%E1%84%86%E1%85%A1%E1%84%91%E1%85%A9%E1%84%80%E1%85%AE_%E1%84%92%E1%85%B3%E1%86%B8%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B5%E1%84%89%E1%85%A5%E1%86%AF%20%E1%84%92%E1%85%A7%E1%86%AB%E1%84%92%E1%85%AA%E1%86%BC_20220418.csv')
mf_a.drop(['Unnamed: 8'], axis=1, inplace=True)
mf_a.head()

,시도명,시군구명,상호명,소재지도로명주소,업종,흡연실구분,흡연시설형태,데이터기준일자
0,서울특별시,마포구,3POP 빈티지 PC CAFE,서울특별시 마포구 월드컵북로30길 13,게임제공업소,실내,밀폐형,44669.0
1,서울특별시,마포구,이룸PC,서울특별시 마포구 마포대로 38,게임제공업소,실내,밀폐형,44669.0
2,서울특별시,마포구,제노PC방(홍대2),서울특별시 마포구 와우산로 83,게임제공업소,실내,밀폐형,44669.0
3,서울특별시,마포구,I PC CAFE 홍대역점,서울특별시 마포구 양화로 156,게임제공업소,실내,밀폐형,44669.0
4,서울특별시,마포구,히어로PC방,서울특별시 마포구 독막로 241,게임제공업소,실내,밀폐형,44669.0


In [15]:
mf_a['소재지도로명주소'] = mf_a['소재지도로명주소'].map(lambda x : x.split('(')[0])
mf_a['소재지도로명주소']

0      서울특별시 마포구 월드컵북로30길 13
1          서울특별시 마포구 마포대로 38
2          서울특별시 마포구 와우산로 83
3          서울특별시 마포구 양화로 156
4          서울특별시 마포구 독막로 241
               ...          
352        서울특별시 마포구 독막로 17 
353     서울특별시 마포구 양화로18안길 12
354      서울특별시 마포구 새창로2길 13 
355     서울특별시 마포구 월드컵로42길 13
356      서울특별시 마포구 월드컵로 212 
Name: 소재지도로명주소, Length: 357, dtype: object

In [16]:
mf_a = mf_a.drop_duplicates(subset='소재지도로명주소')
mf_a['소재지도로명주소'].duplicated()

0      False
1      False
2      False
3      False
4      False
       ...  
351    False
352    False
353    False
354    False
356    False
Name: 소재지도로명주소, Length: 290, dtype: bool

In [17]:
import json
import requests

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + 'key값'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']), float(match_first['x'])

address= mf_t['도로명주소']
print(address.head())

latitude = []
longitude =[]

for i in address:
    try:
        result = addr_to_lat_lon(i)
        latitude.append(result[0])
        longitude.append(result[1])
    except Exception as e:
        print(f"주소 검색 오류: {e}")
        latitude.append(None)  # 또는 다른 기본값 설정
        longitude.append(None)

mf_t['위도'] = latitude
mf_t['경도'] = longitude
mf_t.head()

0     서울특별시 마포구 양화로 156
1     서울특별시 마포구 양화로 183
2    서울특별시 마포구 마포대로 144
3     서울특별시 마포구 상암산로 38
Name: 도로명주소, dtype: object


,시도명,시군구명,도로명주소,세부위치,데이터기준일자,위도,경도
0,서울특별시,마포구,서울특별시 마포구 양화로 156,홍대입구역 9번출구 LG팰리스 뒤쪽 도로,2022-08-05,37.556113,126.923447
1,서울특별시,마포구,서울특별시 마포구 양화로 183,홍대입구역 3번출구 인근,2022-08-05,37.558368,126.925148
2,서울특별시,마포구,서울특별시 마포구 마포대로 144,마포T타운 뒤쪽 도로,2022-08-05,37.547001,126.953757
3,서울특별시,마포구,서울특별시 마포구 상암산로 38,kbs 건물 인근,2022-08-05,37.576614,126.889693


In [18]:
address= mf_a['소재지도로명주소']
print(address.head())

latitude = []
longitude =[]

for i in address:
    try:
        result = addr_to_lat_lon(i)
        latitude.append(result[0])
        longitude.append(result[1])
    except Exception as e:
        print(f"주소 검색 오류: {e}")
        latitude.append(None)  # 또는 다른 기본값 설정
        longitude.append(None)

mf_a['위도'] = latitude
mf_a['경도'] = longitude
mf_a.head()

0    서울특별시 마포구 월드컵북로30길 13
1        서울특별시 마포구 마포대로 38
2        서울특별시 마포구 와우산로 83
3        서울특별시 마포구 양화로 156
4        서울특별시 마포구 독막로 241
Name: 소재지도로명주소, dtype: object
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range
주소 검색 오류: list index out of range


,시도명,시군구명,상호명,소재지도로명주소,업종,흡연실구분,흡연시설형태,데이터기준일자,위도,경도
0,서울특별시,마포구,3POP 빈티지 PC CAFE,서울특별시 마포구 월드컵북로30길 13,게임제공업소,실내,밀폐형,44669.0,37.568595,126.908364
1,서울특별시,마포구,이룸PC,서울특별시 마포구 마포대로 38,게임제공업소,실내,밀폐형,44669.0,37.539427,126.946661
2,서울특별시,마포구,제노PC방(홍대2),서울특별시 마포구 와우산로 83,게임제공업소,실내,밀폐형,44669.0,37.551939,126.923292
3,서울특별시,마포구,I PC CAFE 홍대역점,서울특별시 마포구 양화로 156,게임제공업소,실내,밀폐형,44669.0,37.556113,126.923447
4,서울특별시,마포구,히어로PC방,서울특별시 마포구 독막로 241,게임제공업소,실내,밀폐형,44669.0,NaN,NaN


In [19]:
"""
네이버에 주소랑 업체명 찾아봐도 안나오는 데이터(폐업이라고 추측) 삭제(2개는 중복([231,312],[237,311])),
나오는 2개는([195],[206]) 찾아서 직접 기입
"""
mf_a[mf_a['위도'].isnull()]

,시도명,시군구명,상호명,소재지도로명주소,업종,흡연실구분,흡연시설형태,데이터기준일자,위도,경도
4,서울특별시,마포구,히어로PC방,서울특별시 마포구 독막로 241,게임제공업소,실내,밀폐형,44669.0,NaN,NaN
17,서울특별시,마포구,타임 피시 카페,서울특별시 마포구 만리재옛길 55-1,게임제공업소,실내,밀폐형,44669.0,NaN,NaN
63,서울특별시,마포구,VRIZ PC CAFE,서울특별시 마포구 양화로 136,게임제공업소,실내,밀폐형,44669.0,NaN,NaN
195,서울특별시,마포구,마포도서관,서울특별시 마포구 홍익로2가 16,공장 및 복합건축물,실외,폐쇄형,44669.0,NaN,NaN
206,서울특별시,마포구,서정빌딩,서울특별시 마포구 어울마당로138,공장 및 복합건축물,실외,개방형,44669.0,NaN,NaN
231,서울특별시,마포구,챔프당구장,서울특별시 마포구 만리재옛길 100,실내체육시설,실내,밀폐형,44669.0,NaN,NaN
237,서울특별시,마포구,엠케이당구장,서울특별시 마포구 만리재로 20-5,실내체육시설,실내,밀폐형,44669.0,NaN,NaN
311,서울특별시,마포구,엠케이당구장,서울특별시 마포구 만리재로 20-5,실내체육시설,실내,밀폐형,44669.0,NaN,NaN
312,서울특별시,마포구,챔프당구장,서울특별시 마포구 만리재옛길 100,실내체육시설,실내,밀폐형,44669.0,NaN,NaN


In [20]:
mf_a['위도'][195] = 37.55275613425722
mf_a['경도'][195] = 126.94159333241558
mf_a['위도'][206] = 37.556031211508746
mf_a['경도'][206] = 126.92549740198737

<ipython-input-20-5b3f178ae75e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf_a['위도'][195] = 37.55275613425722
<ipython-input-20-5b3f178ae75e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf_a['경도'][195] = 126.94159333241558
<ipython-input-20-5b3f178ae75e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mf_a['위도'][206] = 37.556031211508746
<ipython-input-20-5b3f178ae75e>:4: SettingWithCopyWarning: 
A value is trying to be se

In [21]:
mf_a = mf_a.dropna(subset=['위도'], how='any')
mf_a

,시도명,시군구명,상호명,소재지도로명주소,업종,흡연실구분,흡연시설형태,데이터기준일자,위도,경도
0,서울특별시,마포구,3POP 빈티지 PC CAFE,서울특별시 마포구 월드컵북로30길 13,게임제공업소,실내,밀폐형,44669.0,37.568595,126.908364
1,서울특별시,마포구,이룸PC,서울특별시 마포구 마포대로 38,게임제공업소,실내,밀폐형,44669.0,37.539427,126.946661
2,서울특별시,마포구,제노PC방(홍대2),서울특별시 마포구 와우산로 83,게임제공업소,실내,밀폐형,44669.0,37.551939,126.923292
3,서울특별시,마포구,I PC CAFE 홍대역점,서울특별시 마포구 양화로 156,게임제공업소,실내,밀폐형,44669.0,37.556113,126.923447
5,서울특별시,마포구,긱스타 PC CAFE,서울특별시 마포구 월드컵로1길 14,게임제공업소,실내,밀폐형,44669.0,37.550050,126.912061
...,...,...,...,...,...,...,...,...,...,...
351,서울특별시,마포구,인버스YTN점,서울특별시 마포구 상암산로 76,음식점,실외,폐쇄형,44669.0,37.579194,126.892315
352,서울특별시,마포구,명륜진사갈비 합정역점,서울특별시 마포구 독막로 17,음식점,실외,개방형,44669.0,37.548752,126.915396
353,서울특별시,마포구,위드커피,서울특별시 마포구 양화로18안길 12,음식점,실외,개방형,44669.0,37.556675,126.925336
354,서울특별시,마포구,도화곱창,서울특별시 마포구 새창로2길 13,음식점,실외,개방형,44669.0,37.541337,126.949445


In [23]:
mf_a['위도'].isnull().sum()

0

In [ ]:
import folium

# 서울시의 위도
latitude = 37.5665350
longitude = 126.9779692

m_seoul = folium.Map(location=[latitude, longitude],
               zoom_start=12, 
               width=1300, 
               height=800
              )
m_seoul

### 🔴 투기지역, 🔵 흡연구역

In [27]:
m_map = mf_a[["위도", "경도"]].dropna()
for lat, long in zip(m_map["위도"], m_map["경도"]):
    folium.CircleMarker([lat, long], radius=23, color='#3186cc', fill_color='#3186cc').add_to(m_seoul)

m_map2 = mf_t[["위도", "경도"]].dropna()
for lat, long in zip(m_map2["위도"], m_map2["경도"]):
    folium.CircleMarker([lat, long], radius=50, color='#cc4831', fill_color='#cc4831').add_to(m_seoul)

m_seoul